In [1]:
import sys
sys.path.append("..")
from model import model
from model import dataset as ds
from torch.utils.data import DataLoader
import os
import torch
from torchvision import transforms
from utils import import_data 
from torchvision.models import resnet50
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [ ]:
images, bboxes, means, stds = import_data(
    annotations_path="../datasets/annotations/",
    images_path="../frames/",
    channels=3
)

In [ ]:
train_dataset = ds.ImageDataset(
    images,
    transforms=transforms.ToTensor(),
    target_transforms=transforms.ToTensor()
)

In [ ]:
len(train_dataset)
train_loader = DataLoader(
    train_dataset,
    batch_size=len(train_dataset.images),
    shuffle=True,
    num_workers=os.cpu_count()
    )

In [ ]:


#TODO Read: https://d2l.ai/chapter_computer-vision/anchor.html


resnet = resnet50(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False

model = model.BoxRegressor(base_model=resnet).to(DEVICE)
bbox_loss_func = torch.nn.MSELoss()
optimizator = torch.optim.Adam(model.parameters(), lr=.01)
train_loss = []

for epoch in range(5):
    model.train()
    loss = 0
    for (images, bboxes) in train_loader:
        (images, bboxes) = (images.to(DEVICE), bboxes.to(DEVICE))
        predictions = model(images)
        bbox_loss = bbox_loss_func(predictions, bboxes)
        loss += bbox_loss

In [16]:
import torch
torch.tensor([(1,2), (3,4), (5,6)]).size()


torch.Size([3, 2])